In [1]:
! ls

README.md                              example.py
__pycache__                            for-colab-seq2seq-lang-model.ipynb
colab_version_seq2seq_lang_model.ipynb language_model.py
data                                   seq2seq-lang-model.ipynb
datasets.py                            seq2seqdataprocessing.py
download_cornell.sh                    training_checkpoints
download_opensubs.sh


## Check GPU 

In [1]:
from tensorflow.python.client import device_lib


[x for x in device_lib.list_local_devices() if x.device_type == 'GPU']

[]

## Clone github repository

In [ ]:
! git clone https://github.com/nabacg/aml-nlp-notes.git

In [ ]:
import os 
print(os.getcwd())
[n for n in dir(os) if 'ch' in n]
os.chdir('/content/aml-nlp-notes/language-model')
print(os.getcwd())

## Download data files

## cornell dataset

In [ ]:
! sh download_cornell.sh

## twitter dataset 

In [ ]:
## manually upload twitter.env.txt

## Open subtitles 

In [ ]:
! sh download_opensubs.sh

## Imports required

In [1]:
import tensorflow as tf
import os
from seq2seqdataprocessing import load_dataset, word_to_idx
from sklearn.model_selection import train_test_split
from language_model import Encoder, Decoder, train_model


1.13.1


## Prepare train, test dataset

In [4]:
dataset_name = 'cornell'
input_tensor, target_tensor, dict_index, max_length_inp, max_length_targ = load_dataset(dataset_name = dataset_name, max_sentence_length=20)

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

100%|██████████| 83097/83097 [00:05<00:00, 15550.44it/s]


(22312, 22312, 2480, 2480)

(22312, 22312, 2480, 2480)

In [5]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
word2idx = dict_index[0]
embedding_dim = 256
units = 1024
vocab_size = len(word2idx)


dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

## Create encoder, decoder instance

In [7]:
encoder = Encoder(vocab_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_size, embedding_dim, units, BATCH_SIZE)
optimizer = tf.train.AdamOptimizer()

## Mount Google drive to save model checkpoints

In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

## Checkpoints 

In [8]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

### Set checkpoint for Google drive 

In [ ]:
checkpoint_dir = "/content/drive/My Drive/NLP-checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

### copy checkpoints from checkpoints_prefix to Google drive mount

In [ ]:
! cp $checkpoint_dir/* /content/drive/'My Drive'/NLP-checkpoints

### Restore previous checkpoints 

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Training 

In [10]:
train_model(encoder, 
            decoder, 
            optimizer,
            dataset, 
            BATCH_SIZE, 
            N_BATCH,
            start_word_index=word_to_idx(word2idx, '<start>'),
            epochs=3,
            checkpoint_prefix=checkpoint_prefix)

TypeError: train_model() got multiple values for argument 'start_word_index'